In [1]:
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
from scipy.sparse import csc_matrix
from get_dataset import get_dataset
from optimizers import Ig, Nesterov, Sgd, Shuffling
from loss import LogisticRegression

# Setup for plotting
sns.set(style="whitegrid", font_scale=1.2, context="talk", palette=sns.color_palette("bright"))
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['figure.figsize'] = (8, 6)

# Get data and set parameters
dataset = 'w8a.txt'
A, b = get_dataset(dataset)
loss = LogisticRegression(A, b, l1=0, l2=0)
n, dim = A.shape
L = loss.smoothness()
l2 = L / np.sqrt(n)
loss.l2 = l2
x0 = csc_matrix((dim, 1))
n_epoch = 600
batch_size = 512
n_seeds = 2  # Set to 20 in the paper
stoch_it = 250 * n // batch_size

# Run the methods and collect results
results = {}

# Nesterov
nest_str = Nesterov(loss=loss, it_max=n_epoch, mu=l2, strongly_convex=True)
results['Nesterov'] = nest_str.run(x0=x0)

# Shuffling with different configurations
rr = Shuffling(loss=loss, lr0=1/l2, lr_max=1/loss.batch_smoothness(batch_size), lr_decay_coef=l2/3,
               it_max=stoch_it, n_seeds=n_seeds, batch_size=batch_size)
results['RR'] = rr.run(x0=x0)

so = Shuffling(loss=loss, lr0=1/l2, lr_max=1/loss.batch_smoothness(batch_size), lr_decay_coef=l2/3,
               it_max=stoch_it, n_seeds=n_seeds, batch_size=batch_size, steps_per_permutation=np.inf)
results['Shuffle-once'] = so.run(x0=x0)

# Stochastic Gradient Descent
sgd = Sgd(loss=loss, lr_max=1/loss.batch_smoothness(batch_size), lr0=1/l2, lr_decay_coef=l2/2,
          it_max=stoch_it, n_seeds=1, batch_size=batch_size, avoid_cache_miss=True)
results['SGD'] = sgd.run(x0=x0)

# Incremental Gradient
ig = Ig(loss=loss, lr0=1/l2, lr_max=1/loss.batch_smoothness(batch_size), lr_decay_coef=l2/3,
        it_max=stoch_it, batch_size=batch_size)
results['IG'] = ig.run(x0=x0)

# Visualization
plt.figure()
for method, result in results.items():
    plt.plot(result['iterations'], result['losses'], label=method)
plt.yscale('log')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.title('Optimization Loss over Iterations')
plt.show()


TypeError: '>' not supported between instances of 'NoneType' and 'int'